![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [400]:
import pandas as pd
import numpy as np

# Start coding here...

In [401]:
df = pd.read_csv("bank_marketing.csv")

## extract and clean data for the client table

In [402]:
client = df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]
print(client.head())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no


education	object	Client's level of education	Change "." to "_" and "unknown" to np.NaN

credit_default	bool	Whether the client's credit is in default	Convert to boolean data type: 1 if "yes", otherwise 0

mortgage	bool	Whether the client has an existing mortgage (housing loan)	Convert to boolean data type: 1 if "yes", otherwise 0

In [403]:
client['education'] = client['education'].str.replace(".", "_")
print(client['education'].head())
client['education'] = client['education'].replace("unknown", np.nan)

0       basic_4y
1    high_school
2    high_school
3       basic_6y
4    high_school
Name: education, dtype: object


In [404]:
# client.replace("unknown", np.nan, inplace = True)
# print(client.isna().sum())

In [405]:
client['credit_default'].info()

mapping_dict = {'unknown': 0, 'no': 0, 'yes': 1}
#change to boolean datatype
client['credit_default'] = client['credit_default'].replace(mapping_dict)
client['credit_default'] = client['credit_default'].astype(bool)


<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: credit_default
Non-Null Count  Dtype 
--------------  ----- 
41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB


In [406]:
mapping_dict = {'unknown': 0, 'no': 0, 'yes': 1}

client['mortgage'] = client['mortgage'].map(mapping_dict).astype(bool)

In [407]:
client['mortgage'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: mortgage
Non-Null Count  Dtype
--------------  -----
41188 non-null  bool 
dtypes: bool(1)
memory usage: 40.3 KB


In [408]:
client.to_csv("client.csv", index = False)

## clean for campaign table

previous_outcome	bool	Outcome of the previous campaign	Convert to boolean data type: 1 if "success", otherwise 0.

campaign_outcome	bool	Outcome of the current campaign	Convert to boolean data type: 1 if "yes", otherwise 0.

last_contact_date	datetime	Last date the client was contacted	Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD"

In [409]:
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome","campaign_outcome"]]
print(campaign.head())

   client_id  number_contacts  ...  previous_outcome  campaign_outcome
0          0                1  ...       nonexistent                no
1          1                1  ...       nonexistent                no
2          2                1  ...       nonexistent                no
3          3                1  ...       nonexistent                no
4          4                1  ...       nonexistent                no

[5 rows x 6 columns]


In [410]:
campaign['previous_outcome'].info()
mapping_dict = {'failure' : 0, 'success' : 1, 'nonexistent' : 0}

campaign['previous_outcome'] = campaign['previous_outcome'].map(mapping_dict)

#change to boolean datatype

campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)
campaign['previous_outcome'].head(20)

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: previous_outcome
Non-Null Count  Dtype 
--------------  ----- 
41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: previous_outcome, dtype: bool

In [411]:
mapping_dict = {"yes":1, "no":0}
campaign['campaign_outcome'] = campaign['campaign_outcome'].map(mapping_dict).astype(bool)
campaign['campaign_outcome'].head()

0    False
1    False
2    False
3    False
4    False
Name: campaign_outcome, dtype: bool

In [412]:
df["year"] = 2022
campaign['last_contact_date'] = pd.to_datetime(df[["year","month","day"]].astype(str).agg('-'.join, axis=1))
print(campaign['last_contact_date'])

0       2022-05-13
1       2022-05-19
2       2022-05-23
3       2022-05-27
4       2022-05-03
           ...    
41183   2022-11-30
41184   2022-11-06
41185   2022-11-24
41186   2022-11-17
41187   2022-11-23
Name: last_contact_date, Length: 41188, dtype: datetime64[ns]


In [413]:
campaign.to_csv("campaign.csv", index = False)

In [414]:
economics = df[["client_id", "cons_price_idx","euribor_three_months"]]
print(economics.head())

   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


In [415]:
economics.to_csv("economics.csv", index = False)